# Issue Credential - Carol
## Role: Holder

### This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [issuer notebook](http://localhost:8888/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb) from Alice's perspective. 


If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)

## First complete steps 1-6 in the issuer notebook.

## 7. Initialise the Controller

In [12]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"
API_KEY = "carol_api_123456789"


agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

IPython autoawait is `on`, and set to use `asyncio`


In [13]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 8. Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [14]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Subscribing too: issue_credential


## 9. Continue in the [Issuer](http://localhost:8888/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb) Notebook

This sends the credential to the holder Bob and should be recognisable through the print statements from the above handler.

## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.

In [15]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'auto_remove': True, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:chloebaba_schema:0.0.1', 'updated_at': '2023-01-02 07:22:51.447778Z', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:chloebaba_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:29:default', 'key_correctness_proof': {'c': '26061038285043942401771372234745453476866948947072369932444368421308209374035', 'xz_cap': '129998306516737812517248153139143322098471685910219220959531098825871709118415589481125927370175833054959562748724973549055246780803287695686618149296227963783299329448181466270170923058033890284225868808480467241850270917139977220127227054710184998167538963860737557101984438820542229227380443779370987830813322664715685735932895547828485798474111337485788944710591544846544216525406275077861325899220786204486794782872363228416320905404062448637271448754558025226162885966358092983924424053290867445016418125637304374490846521093996878135398904658697546949945240882518700229137533749907933

In [16]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: offer_received
Being offered: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]


## 11. Request Credential from Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [17]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: request_sent
Handle Credentials
Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: request_sent
Attributes: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]
Handle Credentials
Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: credential_received
Attributes: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [18]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: credential_received


In [19]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "Creator's Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: credential_acked
Attributes: [{'name': 'author', 'value': 'Alice'}, {'name': 'name', 'value': 'researcher'}, {'name': 'time', 'value': '01-02-2023, 07:20:39'}, {'name': 'is_original', 'value': '1'}]
Credential exchange 1b707214-47c6-4ff7-97ca-1e0d2aa949c0, role: holder, state: credential_acked


## Not Included

* Propose Credential: Allows holder to propose an alternative credential that it wishes the issuer to issue. Most likely used when negotiating the values within the credential or if the holder spots an error in current issued credential. **Note: this generates a new credential exchange id.**



## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [20]:
response = await agent_controller.terminate()
